In [107]:
import pandas as pd
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


In [108]:

drug_count = pd.read_csv('C:/Users/eshiv/code_task_federated/DrugCount.csv')
lab_count = pd.read_csv('C:/Users/eshiv/code_task_federated/LabCount.csv')
members = pd.read_csv('C:/Users/eshiv/code_task_federated/Members.csv')

In [109]:
#import dataset
test_set = pd.read_csv('C:/Users/eshiv/code_task_federated/test_data_client0.csv')

In [110]:
#trainset with DrugCount and LabCount
merged_df = pd.merge(test_set, drug_count, on=['MemberID', 'Year'], how='inner')

# Display the merged DataFrame
print(merged_df)

        MemberID  Vendor    PCP Year   Specialty         PlaceSvc PayDelay  \
0         276553  240043  58880   Y2  Laboratory  Independent Lab       29   
1         276553  240043  58880   Y2  Laboratory  Independent Lab       29   
2         276553  240043  58880   Y2  Laboratory  Independent Lab       29   
3         276553  240043  58880   Y2  Laboratory  Independent Lab       29   
4         276553  240043  58880   Y2  Laboratory  Independent Lab       29   
...          ...     ...    ...  ...         ...              ...      ...   
659725  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
659726  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
659727  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
659728  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
659729  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   

        LengthOfStay PrimaryConditionGroup CharlsonIndex Proced

In [111]:
for col in merged_df.columns:
    if col != 'MemberID':
        unique_values = merged_df[col].unique()
        if len(unique_values) < 20:
            print(f"'{col}': {unique_values}")


'Vendor': [240043]
'Year': ['Y2']
'Specialty': ['Laboratory' nan]
'PlaceSvc': ['Independent Lab' 'Outpatient Hospital' 'Office' nan]
'LengthOfStay': [nan]
'CharlsonIndex': ['1-2' '0' '3-4' '5+']
'ProcedureGroup': ['SCS' 'PL' 'MED' nan]
'ClaimsTruncated': [0 1]
'DaysInHospital': [ 0  1  3  5  4  2  7  6  8 10 15  9 11 14 13 12]
'DaySinceFirstClaim': ['0- 1 month' '8- 9 months' '2- 3 months' '7- 8 months' '1- 2 months'
 '6- 7 months' '10-11 months' '11-12 months' '4- 5 months' '9-10 months'
 '5- 6 months' '3- 4 months' nan]
'LOSSuppression': [0]
'DSFS': ['9-10 months' '2- 3 months' '11-12 months' '10-11 months' '4- 5 months'
 '0- 1 month' '6- 7 months' '5- 6 months' '7- 8 months' '1- 2 months'
 '8- 9 months' '3- 4 months']
'DrugCount': ['3' '4' '5' '2' '1' '6' '7+']


In [112]:
combined_df = pd.merge(merged_df, lab_count, on=['MemberID', 'Year'], how='inner')

# Display the combined DataFrame
print(combined_df)

         MemberID  Vendor    PCP Year   Specialty         PlaceSvc PayDelay  \
0          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
1          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
2          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
3          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
4          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
...           ...     ...    ...  ...         ...              ...      ...   
3857604  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
3857605  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
3857606  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
3857607  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
3857608  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   

         LengthOfStay PrimaryConditionGroup Charlso

In [113]:
combined_df.isnull().sum()

MemberID                       0
Vendor                         0
PCP                            0
Year                           0
Specialty                    183
PlaceSvc                     313
PayDelay                       0
LengthOfStay             3857609
PrimaryConditionGroup        996
CharlsonIndex                  0
ProcedureGroup                18
ClaimsTruncated                0
DaysInHospital                 0
DaySinceFirstClaim          8416
LOSSuppression                 0
DSFS_x                         0
DrugCount                      0
DSFS_y                         0
LabCount                       0
dtype: int64

In [114]:
final_df = pd.merge(combined_df, members, on='MemberID', how='inner')

# Display the final DataFrame
print(final_df)

         MemberID  Vendor    PCP Year   Specialty         PlaceSvc PayDelay  \
0          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
1          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
2          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
3          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
4          276553  240043  58880   Y2  Laboratory  Independent Lab       29   
...           ...     ...    ...  ...         ...              ...      ...   
3857604  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
3857605  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
3857606  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
3857607  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   
3857608  47177983  240043  39372   Y2  Laboratory  Independent Lab       21   

         LengthOfStay PrimaryConditionGroup Charlso

In [115]:
categorical_cols = final_df.select_dtypes(include=['object', 'category']).columns
numerical_cols = final_df.select_dtypes(include=['int', 'float']).columns

# Display the categorical and numerical columns
print("Categorical Columns:")
print(categorical_cols)
print("\nNumerical Columns:")
print(numerical_cols)

Categorical Columns:
Index(['Year', 'Specialty', 'PlaceSvc', 'PayDelay', 'PrimaryConditionGroup',
       'CharlsonIndex', 'ProcedureGroup', 'DaySinceFirstClaim', 'DSFS_x',
       'DrugCount', 'DSFS_y', 'LabCount', 'AgeAtFirstClaim', 'Sex'],
      dtype='object')

Numerical Columns:
Index(['MemberID', 'Vendor', 'PCP', 'LengthOfStay', 'ClaimsTruncated',
       'DaysInHospital', 'LOSSuppression'],
      dtype='object')


In [116]:
final_df.drop(['DSFS_x','DSFS_y'],axis=1,inplace=True)

In [117]:
final_df.drop(['LengthOfStay'],axis=1,inplace=True)final_df.drop(['DSFS_x','DSFS_y'],axis=1,inplace=True)
final_df['Sex'].fillna("Unknown", inplace=True)
final_df['AgeAtFirstClaim'].fillna("Unknown", inplace=True)
final_df[final_df['DaySinceFirstClaim'].isnull() ]

,MemberID,Vendor,PCP,Year,Specialty,PlaceSvc,PayDelay,PrimaryConditionGroup,CharlsonIndex,ProcedureGroup,ClaimsTruncated,DaysInHospital,DaySinceFirstClaim,LOSSuppression,DrugCount,LabCount,AgeAtFirstClaim,Sex
76028,72589470,240043,77134,Y2,Laboratory,Independent Lab,20,GIOBSENT,1-2,SCS,0,0,NaN,0,2,8,30-39,Unknown
76029,72589470,240043,77134,Y2,Laboratory,Independent Lab,20,GIOBSENT,1-2,SCS,0,0,NaN,0,2,7,30-39,Unknown
76030,72589470,240043,77134,Y2,Laboratory,Independent Lab,20,GIOBSENT,1-2,SCS,0,0,NaN,0,2,4,30-39,Unknown
76031,72589470,240043,77134,Y2,Laboratory,Independent Lab,20,GIOBSENT,1-2,SCS,0,0,NaN,0,2,8,30-39,Unknown
76032,72589470,240043,77134,Y2,Laboratory,Independent Lab,20,GIOBSENT,1-2,SCS,0,0,NaN,0,2,7,30-39,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3856510,43083400,240043,18175,Y2,Laboratory,Independent Lab,27,FLaELEC,1-2,PL,0,3,NaN,0,5,1,Unknown,F
3856511,43083400,240043,18175,Y2,Laboratory,Independent Lab,27,FLaELEC,1-2,PL,0,3,NaN,0,5,4,Unknown,F
3856512,43083400,240043,18175,Y2,Laboratory,Independent Lab,27,FLaELEC,1-2,PL,0,3,NaN,0,5,1,Unknown,F
3856795,65219564,240043,34893,Y2,Laboratory,Independent Lab,23,GIOBSENT,0,PL,0,0,NaN,0,2,1,0-9,M


In [118]:
#handling speciality
# final_df[final_df['Specialty'].isnull()]
final_df.dropna(subset=['Specialty'], inplace=True)


In [119]:
#handling PlaceSvc
final_df.dropna(subset=['PlaceSvc'], inplace=True)


In [120]:
#Handling primary condiotion group
final_df.dropna(subset=['PrimaryConditionGroup'], inplace=True)

In [121]:
final_df.dropna(subset=['DaySinceFirstClaim'], inplace=True)

In [122]:
final_df.dropna(subset=['ProcedureGroup'], inplace=True)

In [123]:
final_df.isnull().sum()

MemberID                 0
Vendor                   0
PCP                      0
Year                     0
Specialty                0
PlaceSvc                 0
PayDelay                 0
PrimaryConditionGroup    0
CharlsonIndex            0
ProcedureGroup           0
ClaimsTruncated          0
DaysInHospital           0
DaySinceFirstClaim       0
LOSSuppression           0
DrugCount                0
LabCount                 0
AgeAtFirstClaim          0
Sex                      0
dtype: int64

In [124]:
# Define LabelEncoder`
le = LabelEncoder()

# Encode DrugCount (assuming order matters)
final_df['DrugCount'] = le.fit_transform(final_df['DrugCount'])

# Encode LabCount (assuming order matters)
final_df['LabCount'] = le.fit_transform(final_df['LabCount'])


In [125]:
# Encode PlaceSvc
final_df['PlaceSvc'] = le.fit_transform(final_df['PlaceSvc'])

# Encode CharlsonIndex
final_df['CharlsonIndex'] = le.fit_transform(final_df['CharlsonIndex'])

# Encode ProcedureGroup
final_df['ProcedureGroup'] = le.fit_transform(final_df['ProcedureGroup'])

# Encode DaySinceFirstClaim (consider binning if many categories)
final_df['DaySinceFirstClaim'] = le.fit_transform(final_df['DaySinceFirstClaim'])

# Encode AgeAtFirstClaim (consider binning if many categories)
final_df['AgeAtFirstClaim'] = le.fit_transform(final_df['AgeAtFirstClaim'])

# Encode Sex (consider treating "Unknown" as a separate category)
final_df['Sex'] = le.fit_transform(final_df['Sex'])


In [129]:
final_df.isnull().sum()

MemberID                 0
Vendor                   0
PCP                      0
Year                     0
Specialty                0
PlaceSvc                 0
PayDelay                 0
PrimaryConditionGroup    0
CharlsonIndex            0
ProcedureGroup           0
DaysInHospital           0
DaySinceFirstClaim       0
LOSSuppression           0
DrugCount                0
LabCount                 0
AgeAtFirstClaim          0
Sex                      0
ClaimsTruncated_0        0
ClaimsTruncated_1        0
dtype: int64

In [127]:
# Convert ClaimsTruncated to binary
final_df['ClaimsTruncated_binary'] = final_df['ClaimsTruncated'].map({0: 0, 1: 1})

# One-Hot Encode ClaimsTruncated_binary
ohe = OneHotEncoder(sparse=False)
claims_encoded = ohe.fit_transform(final_df[['ClaimsTruncated_binary']])
claims_cols = ['ClaimsTruncated_0', 'ClaimsTruncated_1']
claims_df = pd.DataFrame(claims_encoded, columns=claims_cols)
final_df = pd.concat([final_df, claims_df], axis=1)

# Drop the original ClaimsTruncated column (optional+)
final_df = final_df.drop('ClaimsTruncated', axis=1)

final_df = final_df.drop('ClaimsTruncated_binary', axis=1)
final_df.dropna(inplace=True)


C:\Users\eshiv\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [132]:
final_df.count()

MemberID                 3838582
Vendor                   3838582
PCP                      3838582
Year                     3838582
Specialty                3838582
PlaceSvc                 3838582
PayDelay                 3838582
PrimaryConditionGroup    3838582
CharlsonIndex            3838582
ProcedureGroup           3838582
DaysInHospital           3838582
DaySinceFirstClaim       3838582
LOSSuppression           3838582
DrugCount                3838582
LabCount                 3838582
AgeAtFirstClaim          3838582
Sex                      3838582
ClaimsTruncated_0        3838582
ClaimsTruncated_1        3838582
dtype: int64

In [133]:
final_df.to_csv('test_0.csv', index=False) 